<a href="https://colab.research.google.com/github/AshvinVignesh/Final_year/blob/main/final_year_DL_lstm_and_sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout,Input , Layer , Bidirectional ,Reshape,MaxPooling2D, Conv1D, MaxPooling1D,Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
import tensorflow.keras.backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Embedding, Dropout, GlobalAveragePooling1D, Flatten, SpatialDropout1D, Bidirectional
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
import pandas as pd
import gensim
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report , f1_score, accuracy_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
df = pd.read_csv("Full_data_aspect_removed_words.csv")
df.head()

,original_text,sentences_with_pos_tags,lang,sentiment,clean_text,Acting,direction,Music,VFX,Genre,excitement,ovr_sent,stop_words
0,இரைச்சல் இல்லாத தெளிவான இசை,இரைச்சல்/NOUN இல்லாத/ADJ தெளிவான/ADJ இசை/NOUN,Tamil,POSITIVE,இரைச்சல் இல்லாத தெளிவான இசை,0,0,1,0,0,0,1,"இல்லாத, தெளிவான"
1,harris romba naal kalichu super song,harris/PROPN romba/ADV naal/NOUN kalichu/VERB ...,Code-Mixed,POSITIVE,harris romba naal kalichu super song,0,0,1,0,0,0,1,"romba, naal, kalichu"
2,hukum pondra iraichal padalai ketu kaathu vali...,hukum/PROPN pondra/ADP iraichal/NOUN padalai/N...,Code-Mixed,NEUTRAL,hukum pondra iraichal padalai ketu kaathu vali...,0,0,1,0,0,0,1,"pondr, iraichal, ketu, kaathu, valichuruchu, a..."
3,youtube la kuda editing nalla pannuvanga,youtube/PROPN la/ADP kuda/ADV editing/NOUN nal...,Code-Mixed,NEUTRAL,youtube kuda editing nalla pannuvanga,0,0,0,1,0,0,1,"la, kuda"
4,amaran is also a diwali release,amaran/PROPN is/AUX also/ADV a/DET diwali/PROP...,Code-Mixed,NEUTRAL,amaran diwali release,0,0,0,0,0,0,0,"is, also, a"


In [ ]:
def prepare_data(df, aspect_column):
    """Prepare data for a specific aspect"""
    # Encode text to numerical values if needed
    # le = LabelEncoder()
    # Changed to return a DataFrame instead of a Series
    X = df[['clean_text']]
    y = df[aspect_column]
    return X, y

In [ ]:
X,y = prepare_data(df,'Acting')

In [ ]:
embeddings_index = {}
f = open('/content/ft_300_fyp_150epoch_window2.txt')
for line in f:
    values = line.split()
    word = values[0] # Since the word is at the 0th index
    coefs = np.asarray(values[1:]) # From the 1st position onwards the values are the embedding values
    embeddings_index[word] = coefs
f.close()

In [ ]:
EMBEDDING_DIM =300

# words not found in embedding index will be all-zeros.

In [ ]:
def analyze_all_aspects_dl(df, aspects, sampling_techniques):
    results = {}
    for aspect in aspects:
        print(f"\nAnalyzing aspect: {aspect}")
        X, y = prepare_data(df, aspect)
        aspect_results = {}
        for technique in sampling_techniques:
            print(f"\nUsing {technique} sampling:")
            mean_score, std_score, reports = train_evaluate_dl_model(X, y, technique)
            aspect_results[technique] = {
                'mean_accuracy': mean_score,
                'std_accuracy': std_score,
                'classification_reports': reports
            }
            print(f"Mean Accuracy: {mean_score:.4f} (\u00b1{std_score:.4f})")
        results[aspect] = aspect_results
    return results

In [ ]:
def train_evaluate_dl_model(X, y, sampling_strategy='none'):
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    scores = []
    reports = []

    # Tokenize and pad sequences
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(X['clean_text'])
    sequences = tokenizer.texts_to_sequences(X['clean_text'])
    padded_sequences = pad_sequences(sequences, maxlen=20)
    word_index = tokenizer.word_index
    num_words = len(word_index) + 1
    embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
    for word, i in word_index.items():
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            # Check if the embedding vector has the expected dimension
            if len(embedding_vector) == EMBEDDING_DIM:
                embedding_matrix[i] = embedding_vector
            else:
                print(f"Warning: Embedding for '{word}' has unexpected dimension {len(embedding_vector)}. Skipping.")
    # Encode labels
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    target_names = [str(class_label) for class_label in le.classes_]
    # Prepare embedding matrix (assuming it's preloaded as `embedding_matrix`)
    vocab_size = len(word_index) + 1

    for fold, (train_idx, val_idx) in enumerate(skf.split(padded_sequences, y_encoded)):
        X_train, X_val = padded_sequences[train_idx], padded_sequences[val_idx]
        y_train, y_val = y_encoded[train_idx], y_encoded[val_idx]

        # Apply undersampling if required
        if sampling_strategy == 'undersample':
            sampler = RandomUnderSampler(random_state=42)
            X_train_flat = X_train.reshape(X_train.shape[0], -1)
            X_train_flat, y_train = sampler.fit_resample(X_train_flat, y_train)
            X_train = X_train_flat.reshape(-1, 20)  # Reshape back
        elif sampling_strategy == 'oversample':
            sampler = SMOTE(random_state=42)
            # Flatten `X_train` for SMOTE
            X_train_flat = X_train.reshape(X_train.shape[0], -1)
            X_train_flat, y_train = sampler.fit_resample(X_train_flat, y_train)
            # Reshape back to the original dimensions
            X_train = X_train_flat.reshape(-1, 20)

        # Define the model
        input_layer = tf.keras.layers.Input(shape=(20,))
        embedding_layer = tf.keras.layers.Embedding(vocab_size, 300, weights=[embedding_matrix], input_length=20, trainable=False)(input_layer)
        # conv = tf.keras.layers.Conv1D(200, kernel_size=3, activation="relu")(embedding_layer)
        lstm = tf.keras.layers.LSTM(128, dropout=0.3, recurrent_dropout=0.2, return_sequences=True)(embedding_layer)
        drop = tf.keras.layers.Dropout(0.2)(lstm)
        dense = tf.keras.layers.Dense(128, activation='relu')(drop)
        flat = tf.keras.layers.Flatten()(dense)
        output_layer = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(flat)

        model = tf.keras.Model(input_layer, output_layer)
        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

        # Train the model
        model.fit(
            X_train, y_train,
            epochs=5,  # You can adjust the number of epochs
            batch_size=32,
            verbose=0
        )

        # Evaluate the model
        y_pred = model.predict(X_val, verbose=0).argmax(axis=1)
        score = accuracy_score(y_val, y_pred)
        scores.append(score)

        # Generate classification report
        report = classification_report(y_val, y_pred, target_names=target_names, output_dict=True)
        reports.append(report)

        print(f"\nFold {fold + 1} Results:")
        print(classification_report(y_val, y_pred, target_names=target_names))

    return np.mean(scores), np.std(scores), reports

In [ ]:
aspects = ['Acting', 'direction', 'Music', 'Genre', 'excitement', 'ovr_sent']
sampling_techniques_under = ['undersample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_under)


Analyzing aspect: Acting

Using undersample sampling:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.05      0.56      0.10        34
           0       0.71      0.25      0.37       770
           1       0.20      0.36      0.26       221

    accuracy                           0.28      1025
   macro avg       0.32      0.39      0.24      1025
weighted avg       0.58      0.28      0.34      1025



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.07      0.55      0.13        33
           0       0.76      0.37      0.50       769
           1       0.24      0.43      0.31       222

    accuracy                           0.39      1024
   macro avg       0.36      0.45      0.31      1024
weighted avg       0.62      0.39      0.44      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.48      0.48      0.48       343
           0       0.28      0.50      0.36       153
           1       0.67      0.51      0.58       528

    accuracy                           0.50      1024
   macro avg       0.48      0.50      0.47      1024
weighted avg       0.55      0.50      0.51      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.50      0.38      0.43       344
           0       0.21      0.61      0.32       152
           1       0.70      0.44      0.54       528

    accuracy                           0.44      1024
   macro avg       0.47      0.47      0.43      1024
weighted avg       0.56      0.44      0.47      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.49      0.53      0.51       344
           0       0.25      0.51      0.33       152
           1       0.63      0.41      0.50       528

    accuracy                           0.47      1024
   macro avg       0.46      0.48      0.45      1024
weighted avg       0.53      0.47      0.48      1024

Mean Accuracy: 0.4759 (±0.0240)


In [ ]:
sampling_techniques_over = ['oversample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_over)


Analyzing aspect: Acting

Using oversample sampling:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.06      0.41      0.10        34
           0       0.81      0.53      0.64       770
           1       0.31      0.41      0.35       221

    accuracy                           0.50      1025
   macro avg       0.39      0.45      0.37      1025
weighted avg       0.68      0.50      0.56      1025



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.05      0.24      0.08        33
           0       0.81      0.54      0.65       769
           1       0.31      0.49      0.38       222

    accuracy                           0.52      1024
   macro avg       0.39      0.42      0.37      1024
weighted avg       0.67      0.52      0.57      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.52      0.42      0.47       343
           0       0.26      0.52      0.35       153
           1       0.69      0.57      0.62       528

    accuracy                           0.51      1024
   macro avg       0.49      0.51      0.48      1024
weighted avg       0.57      0.51      0.53      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.48      0.46      0.47       344
           0       0.28      0.36      0.31       152
           1       0.67      0.63      0.65       528

    accuracy                           0.53      1024
   macro avg       0.48      0.48      0.48      1024
weighted avg       0.55      0.53      0.54      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.52      0.50      0.51       344
           0       0.21      0.61      0.31       152
           1       0.72      0.34      0.46       528

    accuracy                           0.43      1024
   macro avg       0.48      0.48      0.43      1024
weighted avg       0.58      0.43      0.46      1024

Mean Accuracy: 0.4831 (±0.0364)


dravidian W2vec

In [ ]:
aspects = ['Acting', 'direction', 'Music', 'Genre', 'excitement', 'ovr_sent']
sampling_techniques_under = ['undersample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_under)


Analyzing aspect: Acting

Using undersample sampling:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.06      0.35      0.10        34
           0       0.91      0.51      0.65       770
           1       0.45      0.82      0.58       221

    accuracy                           0.57      1025
   macro avg       0.48      0.56      0.45      1025
weighted avg       0.79      0.57      0.62      1025



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.09      0.52      0.16        33
           0       0.91      0.64      0.75       769
           1       0.55      0.73      0.63       222

    accuracy                           0.66      1024
   macro avg       0.52      0.63      0.51      1024
weighted avg       0.80      0.66      0.71      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.72      0.61      0.66       343
           0       0.31      0.53      0.39       153
           1       0.84      0.76      0.80       528

    accuracy                           0.67      1024
   macro avg       0.63      0.63      0.62      1024
weighted avg       0.72      0.67      0.69      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.65      0.72      0.69       344
           0       0.36      0.53      0.43       152
           1       0.86      0.69      0.76       528

    accuracy                           0.67      1024
   macro avg       0.62      0.65      0.63      1024
weighted avg       0.72      0.67      0.69      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.72      0.67      0.70       344
           0       0.28      0.68      0.39       152
           1       0.88      0.55      0.67       528

    accuracy                           0.61      1024
   macro avg       0.63      0.63      0.59      1024
weighted avg       0.74      0.61      0.64      1024

Mean Accuracy: 0.6618 (±0.0291)


In [ ]:
sampling_techniques_over = ['oversample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_over)


Analyzing aspect: Acting

Using oversample sampling:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.11      0.15      0.13        34
           0       0.90      0.89      0.89       770
           1       0.69      0.67      0.68       221

    accuracy                           0.82      1025
   macro avg       0.56      0.57      0.57      1025
weighted avg       0.83      0.82      0.82      1025



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.14      0.18      0.16        33
           0       0.86      0.91      0.89       769
           1       0.68      0.52      0.59       222

    accuracy                           0.80      1024
   macro avg       0.56      0.54      0.55      1024
weighted avg       0.80      0.80      0.80      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.68      0.75      0.71       343
           0       0.40      0.32      0.36       153
           1       0.82      0.81      0.81       528

    accuracy                           0.71      1024
   macro avg       0.63      0.63      0.63      1024
weighted avg       0.71      0.71      0.71      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.66      0.68      0.67       344
           0       0.31      0.48      0.38       152
           1       0.86      0.71      0.78       528

    accuracy                           0.67      1024
   macro avg       0.61      0.62      0.61      1024
weighted avg       0.71      0.67      0.68      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.74      0.75      0.75       344
           0       0.46      0.32      0.37       152
           1       0.77      0.83      0.80       528

    accuracy                           0.73      1024
   macro avg       0.65      0.63      0.64      1024
weighted avg       0.71      0.73      0.72      1024

Mean Accuracy: 0.6934 (±0.0233)


Dravidian fasttext

In [ ]:
aspects = ['Acting', 'direction', 'Music', 'Genre', 'excitement', 'ovr_sent']
sampling_techniques_under = ['undersample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_under)


Analyzing aspect: Acting

Using undersample sampling:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.08      0.38      0.13        34
           0       0.91      0.56      0.69       770
           1       0.45      0.77      0.57       221

    accuracy                           0.60      1025
   macro avg       0.48      0.57      0.46      1025
weighted avg       0.78      0.60      0.65      1025



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.07      0.76      0.13        33
           0       0.90      0.47      0.62       769
           1       0.53      0.67      0.59       222

    accuracy                           0.52      1024
   macro avg       0.50      0.63      0.45      1024
weighted avg       0.79      0.52      0.60      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.70      0.73      0.71       343
           0       0.35      0.48      0.40       153
           1       0.86      0.74      0.79       528

    accuracy                           0.70      1024
   macro avg       0.63      0.65      0.64      1024
weighted avg       0.73      0.70      0.71      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.68      0.71      0.69       344
           0       0.39      0.53      0.45       152
           1       0.85      0.74      0.79       528

    accuracy                           0.70      1024
   macro avg       0.64      0.66      0.64      1024
weighted avg       0.72      0.70      0.71      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.73      0.62      0.67       344
           0       0.31      0.59      0.41       152
           1       0.84      0.70      0.76       528

    accuracy                           0.66      1024
   macro avg       0.63      0.64      0.61      1024
weighted avg       0.72      0.66      0.68      1024

Mean Accuracy: 0.6542 (±0.0413)


In [ ]:
sampling_techniques_over = ['oversample']
results = analyze_all_aspects_dl(df, aspects, sampling_techniques_over)


Analyzing aspect: Acting

Using oversample sampling:


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 1 Results:
              precision    recall  f1-score   support

          -1       0.09      0.12      0.10        34
           0       0.90      0.89      0.89       770
           1       0.68      0.67      0.67       221

    accuracy                           0.81      1025
   macro avg       0.55      0.56      0.55      1025
weighted avg       0.82      0.81      0.82      1025



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 2 Results:
              precision    recall  f1-score   support

          -1       0.10      0.12      0.11        33
           0       0.86      0.87      0.87       769
           1       0.62      0.58      0.60       222

    accuracy                           0.78      1024
   macro avg       0.53      0.52      0.52      1024
weighted avg       0.79      0.78      0.78      1024



...


Fold 3 Results:
              precision    recall  f1-score   support

          -1       0.81      0.58      0.67       343
           0       0.34      0.46      0.39       153
           1       0.77      0.83      0.80       528

    accuracy                           0.69      1024
   macro avg       0.64      0.63      0.62      1024
weighted avg       0.72      0.69      0.70      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 4 Results:
              precision    recall  f1-score   support

          -1       0.64      0.75      0.69       344
           0       0.39      0.28      0.33       152
           1       0.80      0.78      0.79       528

    accuracy                           0.69      1024
   macro avg       0.61      0.60      0.60      1024
weighted avg       0.69      0.69      0.69      1024



/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(



Fold 5 Results:
              precision    recall  f1-score   support

          -1       0.69      0.80      0.74       344
           0       0.41      0.31      0.35       152
           1       0.80      0.77      0.79       528

    accuracy                           0.71      1024
   macro avg       0.63      0.63      0.63      1024
weighted avg       0.71      0.71      0.71      1024

Mean Accuracy: 0.6989 (±0.0084)


#NO  SAMPLING

In [ ]:
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

In [ ]:
tf.device('/device:GPU:0')

In [ ]:
def train_final_dl_models_from_csv(train_df, test_df, aspects):
    """
    For each aspect, loads training data from train_df and testing data from test_df.
    Performs 5-fold cross validation on the training data to select the best deep learning model.
    Evaluates the best model on both the training and test sets and prints classification reports.

    Returns:
        best_models: dict of best models (with their label encoder and tokenizer) for each aspect.
        train_reports: dict of training classification reports for each aspect.
        test_reports: dict of test classification reports for each aspect.
    """
    best_models = {}
    train_reports = {}
    test_reports = {}
    maxlen = 15  # Maximum sequence length

    for aspect in aspects:
        print(f"\n=== Processing aspect: {aspect} ===")
        # Prepare the data for the current aspect
        X_train, y_train = prepare_data(train_df, aspect)
        X_test, y_test = prepare_data(test_df, aspect)

        # Encode target labels
        le = LabelEncoder()
        y_train_encoded = le.fit_transform(y_train)
        y_test_encoded = le.transform(y_test)  # Use the same encoder for test data
        target_names = [str(label) for label in le.classes_]

        # Tokenize the training texts and pad sequences
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(X_train['clean_text'])
        train_sequences = tokenizer.texts_to_sequences(X_train['clean_text'])
        test_sequences = tokenizer.texts_to_sequences(X_test['clean_text'])
        X_train_padded = pad_sequences(train_sequences, maxlen=maxlen)
        X_test_padded = pad_sequences(test_sequences, maxlen=maxlen)

        # Prepare the embedding matrix using the tokenizer's word index
        word_index = tokenizer.word_index
        num_words = len(word_index) + 1
        embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                if len(embedding_vector) == EMBEDDING_DIM:
                    embedding_matrix[i] = embedding_vector
                else:
                    print(f"Warning: Embedding for '{word}' has unexpected dimension {len(embedding_vector)}. Skipping.")
        vocab_size = num_words

        # Cross-validation on the training data to select the best model
        skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        best_cv_score = -np.inf
        best_model = None

        for fold, (cv_train_idx, cv_val_idx) in enumerate(skf.split(X_train_padded, y_train_encoded)):
            X_cv_train = X_train_padded[cv_train_idx]
            y_cv_train = y_train_encoded[cv_train_idx]
            X_cv_val = X_train_padded[cv_val_idx]
            y_cv_val = y_train_encoded[cv_val_idx]

            # Define the model architecture
            input_layer = tf.keras.layers.Input(shape=(maxlen,))
            embedding_layer = tf.keras.layers.Embedding(
                vocab_size,
                EMBEDDING_DIM,
                weights=[embedding_matrix],
                #input_length=maxlen,
                trainable=False
            )(input_layer)
            lstm = tf.keras.layers.LSTM(128, dropout=0.3, recurrent_dropout=0.2, return_sequences=True)(embedding_layer)
            drop = tf.keras.layers.Dropout(0.2)(lstm)
            dense = tf.keras.layers.Dense(128, activation='relu',kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01))(drop)
            flat = tf.keras.layers.Flatten()(dense)
            output_layer = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(flat)

            model = tf.keras.Model(input_layer, output_layer)
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

            # Train the model on the current CV fold
            model.fit(X_cv_train, y_cv_train, epochs=5, batch_size=32, verbose=0)

            # Evaluate on the CV validation fold
            y_cv_pred = model.predict(X_cv_val, verbose=0).argmax(axis=1)
            cv_score = accuracy_score(y_cv_val, y_cv_pred)
            print(f"Fold {fold + 1} CV Accuracy: {cv_score:.4f}")

            if cv_score > best_cv_score:
                best_cv_score = cv_score
                best_model = model

        # Evaluate the best model on the entire training set
        y_train_pred = best_model.predict(X_train_padded, verbose=0).argmax(axis=1)
        train_acc = accuracy_score(y_train_encoded, y_train_pred)
        train_report = classification_report(y_train_encoded, y_train_pred, target_names=target_names)
        print(f"Best CV model Training Accuracy: {train_acc:.4f}")
        print("Training Classification Report:\n", train_report)

        # Evaluate on the test set
        y_test_pred = best_model.predict(X_test_padded, verbose=0).argmax(axis=1)
        test_acc = accuracy_score(y_test_encoded, y_test_pred)
        test_report = classification_report(y_test_encoded, y_test_pred, target_names=target_names)
        print(f"Test Accuracy: {test_acc:.4f}")
        print("Test Classification Report:\n", test_report)

        best_models[aspect.lower()] = {
            "model": best_model,
            "label_encoder": le,
            "tokenizer": tokenizer
        }
        train_reports[aspect.lower()] = train_report
        test_reports[aspect.lower()] = test_report

    return best_models, train_reports, test_reports


In [ ]:
import pickle
import os

def save_models(models, filename="final_models.pkl"):
    """Save trained models and label encoders to a pickle file."""
    with open(filename, 'wb') as f:
        pickle.dump(models, f)
    print(f"Models saved to {filename}")

def load_models(filename="final_models.pkl"):
    """Load trained models and label encoders from a pickle file."""
    if not os.path.exists(filename):
        raise FileNotFoundError(f"The file {filename} does not exist.")
    with open(filename, 'rb') as f:
        models = pickle.load(f)
    print(f"Models loaded from {filename}")
    return models

def predict_all_aspects_from_file(sentence, model_filename="final_models.pkl"):
    """Load the saved models and predict aspects for a given sentence."""
    models = load_models(model_filename)
    predictions = {}

    for aspect, model_info in models.items():
        model = model_info['model']
        le = model_info['label_encoder']
        pred_encoded = model.predict([sentence])[0]
        pred_label = le.inverse_transform([pred_encoded])[0]
        predictions[aspect] = pred_label

    return predictions

In [ ]:
aspects = ['Acting', 'direction', 'Music', 'Genre', 'excitement', 'ovr_sent']

##without regularisation


In [ ]:
best_models, train_reports, test_reports = train_final_dl_models_from_csv(train_df, test_df, aspects)


=== Processing aspect: Acting ===


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Fold 1 CV Accuracy: 0.8238


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Fold 2 CV Accuracy: 0.8452


...

Fold 3 CV Accuracy: 0.6929


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Fold 4 CV Accuracy: 0.7183


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Fold 5 CV Accuracy: 0.7284
Best CV model Training Accuracy: 0.8226
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.75      0.92      0.83      1346
           0       0.72      0.53      0.61       567
           1       0.91      0.84      0.87      2028

    accuracy                           0.82      3941
   macro avg       0.79      0.76      0.77      3941
weighted avg       0.83      0.82      0.82      3941

Test Accuracy: 0.6978
Test Classification Report:
               precision    recall  f1-score   support

          -1       0.70      0.73      0.71       348
           0       0.37      0.35      0.36       139
           1       0.78      0.78      0.78       499

    accuracy                           0.70       986
   macro avg       0.62      0.62      0.62       986
weighted avg       0.70      0.70      0.70       986



##with reguralisation l2


In [ ]:
best_models, train_reports, test_reports = train_final_dl_models_from_csv(train_df, test_df, aspects)


=== Processing aspect: Acting ===
Fold 1 CV Accuracy: 0.8365
Fold 2 CV Accuracy: 0.8325
Fold 3 CV Accuracy: 0.8388
Fold 4 CV Accuracy: 0.8515
Fold 5 CV Accuracy: 0.8350
Best CV model Training Accuracy: 0.8648
Training Classification Report:
               precision    recall  f1-score   support

          -1       1.00      0.01      0.02       131
           0       0.86      0.98      0.92      2962
           1       0.88      0.59      0.71       848

    accuracy                           0.86      3941
   macro avg       0.91      0.53      0.55      3941
weighted avg       0.87      0.86      0.84      3941



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test Accuracy: 0.8418
Test Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00        34
           0       0.85      0.97      0.91       746
           1       0.81      0.50      0.62       206

    accuracy                           0.84       986
   macro avg       0.55      0.49      0.51       986
weighted avg       0.81      0.84      0.82       986


=== Processing aspect: direction ===
Fold 1 CV Accuracy: 0.8390
Fold 2 CV Accuracy: 0.8249
Fold 3 CV Accuracy: 0.8426
Fold 4 CV Accuracy: 0.8109
Fold 5 CV Accuracy: 0.8096
Best CV model Training Accuracy: 0.8615
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.76      0.44      0.56       568
           0       0.87      0.98      0.92      3128
           1       0.79      0.33      0.47       245

    accuracy                           0.86      3941
   macro avg       0.81      0.58      0.65      3

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best CV model Training Accuracy: 0.9609
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00        37
           0       0.96      1.00      0.98      3752
           1       0.79      0.30      0.43       152

    accuracy                           0.96      3941
   macro avg       0.58      0.43      0.47      3941
weighted avg       0.95      0.96      0.95      3941



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test Accuracy: 0.9473
Test Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00        16
           0       0.95      1.00      0.97       926
           1       0.79      0.25      0.38        44

    accuracy                           0.95       986
   macro avg       0.58      0.42      0.45       986
weighted avg       0.93      0.95      0.93       986


=== Processing aspect: excitement ===
Fold 1 CV Accuracy: 0.6439
Fold 2 CV Accuracy: 0.6916
Fold 3 CV Accuracy: 0.6853
Fold 4 CV Accuracy: 0.6815
Fold 5 CV Accuracy: 0.6916
Best CV model Training Accuracy: 0.7546
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.62      0.68      0.65       595
           0       0.76      0.88      0.82      2243
           1       0.86      0.54      0.66      1103

    accuracy                           0.75      3941
   macro avg       0.75      0.70      0.71      

## With l1 and l2

In [ ]:
best_models, train_reports, test_reports = train_final_dl_models_from_csv(train_df, test_df, aspects)


=== Processing aspect: Acting ===
Fold 1 CV Accuracy: 0.8226
Fold 2 CV Accuracy: 0.8325
Fold 3 CV Accuracy: 0.8350
Fold 4 CV Accuracy: 0.8350
Fold 5 CV Accuracy: 0.8452


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best CV model Training Accuracy: 0.8546
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00       131
           0       0.87      0.96      0.91      2962
           1       0.80      0.61      0.69       848

    accuracy                           0.85      3941
   macro avg       0.56      0.52      0.54      3941
weighted avg       0.82      0.85      0.83      3941



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test Accuracy: 0.8306
Test Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00        34
           0       0.84      0.96      0.90       746
           1       0.75      0.49      0.59       206

    accuracy                           0.83       986
   macro avg       0.53      0.48      0.50       986
weighted avg       0.80      0.83      0.80       986


=== Processing aspect: direction ===
Fold 1 CV Accuracy: 0.8162
Fold 2 CV Accuracy: 0.8198
Fold 3 CV Accuracy: 0.8223
Fold 4 CV Accuracy: 0.8135
Fold 5 CV Accuracy: 0.7995


...

Test Accuracy: 0.9493
Test Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00        18
           0       0.97      0.99      0.98       867
           1       0.77      0.76      0.77       101

    accuracy                           0.95       986
   macro avg       0.58      0.58      0.58       986
weighted avg       0.93      0.95      0.94       986


=== Processing aspect: Genre ===
Fold 1 CV Accuracy: 0.9518
Fold 2 CV Accuracy: 0.9530
Fold 3 CV Accuracy: 0.9518
Fold 4 CV Accuracy: 0.9518
Fold 5 CV Accuracy: 0.9518


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Best CV model Training Accuracy: 0.9520
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00        37
           0       0.95      1.00      0.98      3752
           1       0.00      0.00      0.00       152

    accuracy                           0.95      3941
   macro avg       0.32      0.33      0.33      3941
weighted avg       0.91      0.95      0.93      3941



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Test Accuracy: 0.9391
Test Classification Report:
               precision    recall  f1-score   support

          -1       0.00      0.00      0.00        16
           0       0.94      1.00      0.97       926
           1       0.00      0.00      0.00        44

    accuracy                           0.94       986
   macro avg       0.31      0.33      0.32       986
weighted avg       0.88      0.94      0.91       986


=== Processing aspect: excitement ===
Fold 1 CV Accuracy: 0.6768
Fold 2 CV Accuracy: 0.6840
Fold 3 CV Accuracy: 0.6777
Fold 4 CV Accuracy: 0.6662
Fold 5 CV Accuracy: 0.6662
Best CV model Training Accuracy: 0.7171
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.68      0.48      0.56       595
           0       0.70      0.91      0.79      2243
           1       0.84      0.45      0.59      1103

    accuracy                           0.72      3941
   macro avg       0.74      0.61      0.64      

In [ ]:
save_models(best_models)  # Call this after training to save models


Models saved to final_models.pkl


In [ ]:
def predict_all_aspects_from_file(sentence, model_filename="final_models.pkl", maxlen=15): # Added maxlen
    """Load the saved models and predict aspects for a given sentence."""
    models = load_models(model_filename)
    predictions = {}

    for aspect, model_info in models.items():
        model = model_info['model']
        le = model_info['label_encoder']
        tokenizer = model_info['tokenizer']  # Get the tokenizer
        # Tokenize and pad the sentence
        sequence = tokenizer.texts_to_sequences([sentence])
        padded_sequence = pad_sequences(sequence, maxlen=maxlen) # Pad the sequence
        # Now predict using the padded sequence
        pred_probs = model.predict(padded_sequence, verbose=0) # Use padded_sequence for prediction
        pred_encoded = pred_probs.argmax(axis=1)[0]
        pred_label = le.inverse_transform([pred_encoded])[0]
        predictions[aspect] = pred_label

    return predictions


In [ ]:
predictions = predict_all_aspects_from_file("i did not like the movie ")

Models loaded from final_models.pkl


In [ ]:
predictions

{'acting': 0,
 'direction': 0,
 'music': 0,
 'genre': 0,
 'excitement': 0,
 'ovr_sent': 0}

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
train_df = pd.read_csv("train_data.csv")
test_df = pd.read_csv("test_data.csv")

# Ensure EMBEDDING_DIM and embeddings_index are defined in your context

def train_final_dl_models_from_csv(train_df, test_df, aspects):
    """
    For each aspect, loads training data from train_df and testing data from test_df.
    Performs 5-fold cross validation on the training data to select the best deep learning model,
    averaging the training accuracy across folds and using class weights to address imbalanced datasets.
    Evaluates the best model on both the training and test sets and prints classification reports.

    Returns:
        best_models: dict of best models (with their label encoder and tokenizer) for each aspect.
        train_reports: dict of training classification reports for each aspect.
        test_reports: dict of test classification reports for each aspect.
    """
    best_models = {}
    train_reports = {}
    test_reports = {}
    maxlen = 15  # Maximum sequence length

    for aspect in aspects:
        print(f"\n=== Processing aspect: {aspect} ===")
        # Prepare the data for the current aspect
        X_train, y_train = prepare_data(train_df, aspect)
        X_test, y_test = prepare_data(test_df, aspect)

        # Encode target labels
        le = LabelEncoder()
        y_train_encoded = le.fit_transform(y_train)
        y_test_encoded = le.transform(y_test)  # Use the same encoder for test data
        target_names = [str(label) for label in le.classes_]

        # Tokenize the training texts and pad sequences
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(X_train['clean_text'])
        train_sequences = tokenizer.texts_to_sequences(X_train['clean_text'])
        test_sequences = tokenizer.texts_to_sequences(X_test['clean_text'])
        X_train_padded = pad_sequences(train_sequences, maxlen=maxlen)
        X_test_padded = pad_sequences(test_sequences, maxlen=maxlen)

        # Prepare the embedding matrix using the tokenizer's word index
        word_index = tokenizer.word_index
        num_words = len(word_index) + 1
        embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                if len(embedding_vector) == EMBEDDING_DIM:
                    embedding_matrix[i] = embedding_vector
                else:
                    print(f"Warning: Embedding for '{word}' has unexpected dimension {len(embedding_vector)}. Skipping.")
        vocab_size = num_words

        # Cross-validation on the training data to select the best model
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        best_cv_score = -np.inf
        best_model = None
        cv_train_scores = []  # To store training accuracies for each fold

        for fold, (cv_train_idx, cv_val_idx) in enumerate(skf.split(X_train_padded, y_train_encoded)):
            X_cv_train = X_train_padded[cv_train_idx]
            y_cv_train = y_train_encoded[cv_train_idx]
            X_cv_val = X_train_padded[cv_val_idx]
            y_cv_val = y_train_encoded[cv_val_idx]

            # Compute class weights to address imbalance
            classes = np.unique(y_cv_train)
            cw = compute_class_weight(class_weight='balanced', classes=classes, y=y_cv_train)
            class_weight = dict(zip(classes, cw))

            # Define the model architecture
            input_layer = tf.keras.layers.Input(shape=(maxlen,))
            embedding_layer = tf.keras.layers.Embedding(
                vocab_size,
                EMBEDDING_DIM,
                weights=[embedding_matrix],
                trainable=False
            )(input_layer)
            lstm = tf.keras.layers.LSTM(128, dropout=0.3, recurrent_dropout=0.2, return_sequences=True)(embedding_layer)
            drop = tf.keras.layers.Dropout(0.2)(lstm)
            dense = tf.keras.layers.Dense(128, activation='relu',
                                          kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01))(drop)
            flat = tf.keras.layers.Flatten()(dense)
            output_layer = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(flat)

            model = tf.keras.Model(input_layer, output_layer)
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

            # Train the model on the current CV fold using class weights
            model.fit(X_cv_train, y_cv_train, epochs=5, batch_size=32, verbose=0, class_weight=class_weight)

            # Evaluate on the CV training fold for average training score
            y_cv_train_pred = model.predict(X_cv_train, verbose=0).argmax(axis=1)
            fold_train_accuracy = accuracy_score(y_cv_train, y_cv_train_pred)
            cv_train_scores.append(fold_train_accuracy)

            # Evaluate on the CV validation fold
            y_cv_val_pred = model.predict(X_cv_val, verbose=0).argmax(axis=1)
            cv_score = accuracy_score(y_cv_val, y_cv_val_pred)
            print(f"Fold {fold + 1} CV Accuracy: {cv_score:.4f}, Training Accuracy: {fold_train_accuracy:.4f}")

            # Select the best model based on CV validation accuracy
            if cv_score > best_cv_score:
                best_cv_score = cv_score
                best_model = model

        avg_cv_train_score = np.mean(cv_train_scores)
        print(f"Average Training Accuracy across folds: {avg_cv_train_score:.4f}")

        # Evaluate the best model on the entire training set
        y_train_pred = best_model.predict(X_train_padded, verbose=0).argmax(axis=1)
        train_acc = accuracy_score(y_train_encoded, y_train_pred)
        train_report = classification_report(y_train_encoded, y_train_pred, target_names=target_names)
        print(f"Best CV model Training Accuracy on full training set: {train_acc:.4f}")
        print("Training Classification Report:\n", train_report)

        # Evaluate on the test set
        y_test_pred = best_model.predict(X_test_padded, verbose=0).argmax(axis=1)
        test_acc = accuracy_score(y_test_encoded, y_test_pred)
        test_report = classification_report(y_test_encoded, y_test_pred, target_names=target_names)
        print(f"Test Accuracy: {test_acc:.4f}")
        print("Test Classification Report:\n", test_report)

        best_models[aspect.lower()] = {
            "model": best_model,
            "label_encoder": le,
            "tokenizer": tokenizer
        }
        train_reports[aspect.lower()] = train_report
        test_reports[aspect.lower()] = test_report

    return best_models, train_reports, test_reports


In [ ]:
aspects = ['Acting', 'direction', 'Music','ovr_sent']

In [ ]:
best_models, train_reports, test_reports = train_final_dl_models_from_csv(train_df,test_df, aspects)


=== Processing aspect: Acting ===
Fold 1 CV Accuracy: 0.6380, Training Accuracy: 0.6387
Fold 2 CV Accuracy: 0.5888, Training Accuracy: 0.6298
Fold 3 CV Accuracy: 0.6269, Training Accuracy: 0.6972
Fold 4 CV Accuracy: 0.6751, Training Accuracy: 0.7113
Fold 5 CV Accuracy: 0.5305, Training Accuracy: 0.5703
Fold 6 CV Accuracy: 0.5152, Training Accuracy: 0.5718
Fold 7 CV Accuracy: 0.7056, Training Accuracy: 0.7167
Fold 8 CV Accuracy: 0.6954, Training Accuracy: 0.7387
Fold 9 CV Accuracy: 0.6751, Training Accuracy: 0.6704
Fold 10 CV Accuracy: 0.6320, Training Accuracy: 0.6648
Average Training Accuracy across folds: 0.6610
Best CV model Training Accuracy on full training set: 0.7156
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.22      0.68      0.34       131
           0       0.92      0.69      0.79      2962
           1       0.51      0.81      0.63       848

    accuracy                           0.72      3941
   macro av

In [ ]:
def prepare_data(df, aspect_column):
    """Prepare data for a specific aspect"""
    # Encode text to numerical values if needed
    # le = LabelEncoder()
    # Changed to return a DataFrame instead of a Series
    X = df[['R_clean_text']]
    y = df[aspect_column]
    return X, y

In [ ]:
train_df = pd.read_csv("Romanised_train_data.csv")
test_df = pd.read_csv("Romanised_test_data.csv")

# Ensure EMBEDDING_DIM and embeddings_index are defined in your context

def train_final_dl_models_from_csv(train_df, test_df, aspects):
    """
    For each aspect, loads training data from train_df and testing data from test_df.
    Performs 5-fold cross validation on the training data to select the best deep learning model,
    averaging the training accuracy across folds and using class weights to address imbalanced datasets.
    Evaluates the best model on both the training and test sets and prints classification reports.

    Returns:
        best_models: dict of best models (with their label encoder and tokenizer) for each aspect.
        train_reports: dict of training classification reports for each aspect.
        test_reports: dict of test classification reports for each aspect.
    """
    best_models = {}
    train_reports = {}
    test_reports = {}
    maxlen = 15  # Maximum sequence length

    for aspect in aspects:
        print(f"\n=== Processing aspect: {aspect} ===")
        # Prepare the data for the current aspect
        X_train, y_train = prepare_data(train_df, aspect)
        X_test, y_test = prepare_data(test_df, aspect)

        # Encode target labels
        le = LabelEncoder()
        y_train_encoded = le.fit_transform(y_train)
        y_test_encoded = le.transform(y_test)  # Use the same encoder for test data
        target_names = [str(label) for label in le.classes_]

        # Tokenize the training texts and pad sequences
        tokenizer = Tokenizer()
        tokenizer.fit_on_texts(X_train['R_clean_text'])
        train_sequences = tokenizer.texts_to_sequences(X_train['R_clean_text'])
        test_sequences = tokenizer.texts_to_sequences(X_test['R_clean_text'])
        X_train_padded = pad_sequences(train_sequences, maxlen=maxlen)
        X_test_padded = pad_sequences(test_sequences, maxlen=maxlen)

        # Prepare the embedding matrix using the tokenizer's word index
        word_index = tokenizer.word_index
        num_words = len(word_index) + 1
        embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
        for word, i in word_index.items():
            embedding_vector = embeddings_index.get(word)
            if embedding_vector is not None:
                if len(embedding_vector) == EMBEDDING_DIM:
                    embedding_matrix[i] = embedding_vector
                else:
                    print(f"Warning: Embedding for '{word}' has unexpected dimension {len(embedding_vector)}. Skipping.")
        vocab_size = num_words

        # Cross-validation on the training data to select the best model
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        best_cv_score = -np.inf
        best_model = None
        cv_train_scores = []  # To store training accuracies for each fold

        for fold, (cv_train_idx, cv_val_idx) in enumerate(skf.split(X_train_padded, y_train_encoded)):
            X_cv_train = X_train_padded[cv_train_idx]
            y_cv_train = y_train_encoded[cv_train_idx]
            X_cv_val = X_train_padded[cv_val_idx]
            y_cv_val = y_train_encoded[cv_val_idx]

            # Compute class weights to address imbalance
            classes = np.unique(y_cv_train)
            cw = compute_class_weight(class_weight='balanced', classes=classes, y=y_cv_train)
            class_weight = dict(zip(classes, cw))

            # Define the model architecture
            input_layer = tf.keras.layers.Input(shape=(maxlen,))
            embedding_layer = tf.keras.layers.Embedding(
                vocab_size,
                EMBEDDING_DIM,
                weights=[embedding_matrix],
                trainable=False
            )(input_layer)
            lstm = tf.keras.layers.LSTM(128, dropout=0.3, recurrent_dropout=0.2, return_sequences=True)(embedding_layer)
            drop = tf.keras.layers.Dropout(0.2)(lstm)
            dense = tf.keras.layers.Dense(128, activation='relu',
                                          kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01))(drop)
            flat = tf.keras.layers.Flatten()(dense)
            output_layer = tf.keras.layers.Dense(len(le.classes_), activation='softmax')(flat)

            model = tf.keras.Model(input_layer, output_layer)
            model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

            # Train the model on the current CV fold using class weights
            model.fit(X_cv_train, y_cv_train, epochs=5, batch_size=32, verbose=0, class_weight=class_weight)

            # Evaluate on the CV training fold for average training score
            y_cv_train_pred = model.predict(X_cv_train, verbose=0).argmax(axis=1)
            fold_train_accuracy = accuracy_score(y_cv_train, y_cv_train_pred)
            cv_train_scores.append(fold_train_accuracy)

            # Evaluate on the CV validation fold
            y_cv_val_pred = model.predict(X_cv_val, verbose=0).argmax(axis=1)
            cv_score = accuracy_score(y_cv_val, y_cv_val_pred)
            print(f"Fold {fold + 1} CV Accuracy: {cv_score:.4f}, Training Accuracy: {fold_train_accuracy:.4f}")

            # Select the best model based on CV validation accuracy
            if cv_score > best_cv_score:
                best_cv_score = cv_score
                best_model = model

        avg_cv_train_score = np.mean(cv_train_scores)
        print(f"Average Training Accuracy across folds: {avg_cv_train_score:.4f}")

        # Evaluate the best model on the entire training set
        y_train_pred = best_model.predict(X_train_padded, verbose=0).argmax(axis=1)
        train_acc = accuracy_score(y_train_encoded, y_train_pred)
        train_report = classification_report(y_train_encoded, y_train_pred, target_names=target_names)
        print(f"Best CV model Training Accuracy on full training set: {train_acc:.4f}")
        print("Training Classification Report:\n", train_report)

        # Evaluate on the test set
        y_test_pred = best_model.predict(X_test_padded, verbose=0).argmax(axis=1)
        test_acc = accuracy_score(y_test_encoded, y_test_pred)
        test_report = classification_report(y_test_encoded, y_test_pred, target_names=target_names)
        print(f"Test Accuracy: {test_acc:.4f}")
        print("Test Classification Report:\n", test_report)

        best_models[aspect.lower()] = {
            "model": best_model,
            "label_encoder": le,
            "tokenizer": tokenizer
        }
        train_reports[aspect.lower()] = train_report
        test_reports[aspect.lower()] = test_report

    return best_models, train_reports, test_reports


In [ ]:
best_models, train_reports, test_reports = train_final_dl_models_from_csv(train_df,test_df, aspects)


=== Processing aspect: Acting ===
Fold 1 CV Accuracy: 0.7696, Training Accuracy: 0.7479
Fold 2 CV Accuracy: 0.5178, Training Accuracy: 0.5808
Fold 3 CV Accuracy: 0.6624, Training Accuracy: 0.7246
Fold 4 CV Accuracy: 0.6929, Training Accuracy: 0.7480
Fold 5 CV Accuracy: 0.6954, Training Accuracy: 0.7291
Fold 6 CV Accuracy: 0.4898, Training Accuracy: 0.5035
Fold 7 CV Accuracy: 0.5761, Training Accuracy: 0.5678
Fold 8 CV Accuracy: 0.4975, Training Accuracy: 0.5672
Fold 9 CV Accuracy: 0.5914, Training Accuracy: 0.5974
Fold 10 CV Accuracy: 0.5736, Training Accuracy: 0.6197
Average Training Accuracy across folds: 0.6386
Best CV model Training Accuracy on full training set: 0.7501
Training Classification Report:
               precision    recall  f1-score   support

          -1       0.17      0.61      0.27       131
           0       0.89      0.77      0.83      2962
           1       0.64      0.70      0.67       848

    accuracy                           0.75      3941
   macro av

In [ ]:
save_models(best_models)

Models saved to final_models.pkl
